# HackEPS 2022 tests

## Challenge 1 parser

In [68]:
import argparse
import json
from functools import reduce

In [69]:
file_path = "./challenge1"

In [70]:
flatten = lambda l : [item for sublist in l for item in sublist]

### Read and open files

In [71]:
# input reading
inJSON = open(file_path + "/" + "challenge_1_input.json")
data = json.load(inJSON)

# opening/creating output file
plInput = open("./challenge1/prolog_input_challenge_1.pl", "w")

In [72]:
print(data)

{'orders': [{'name': 'charmander_figure::client_4710', 'tasks': [{'machine': 'print_3d', 'duration': 5}, {'machine': 'paint', 'duration': 2}]}, {'name': 'marketing_pin::client_0123', 'tasks': [{'machine': 'print_3d', 'duration': 4}]}, {'name': 'finish_order_412::client_3122', 'tasks': [{'machine': 'assembler', 'duration': 6}, {'machine': 'paint', 'duration': 2}, {'machine': 'print_3d', 'duration': 1}]}, {'name': 'model_suite_prototype::client_8692', 'tasks': [{'machine': 'print_3d', 'duration': 4}, {'machine': 'assembler', 'duration': 3}]}], 'machines': [{'id': 'paint'}, {'id': 'print_3d'}, {'id': 'assembler'}]}


### Order data

In [73]:
machines = {}

mac_to_id = {} #Map a machine name to its id
id_to_mac = {} #Map a machine's id to its name

for idm in range(len(data.get("machines"))):
    m = data.get("machines")[idm]
    machines[m.get('id')] = []
    mac_to_id[m.get('id')] = "m" + str(idm)
    id_to_mac["m" + str(idm)] = m.get('id')


In [74]:
print(machines)

{'paint': [], 'print_3d': [], 'assembler': []}


In [75]:
orders = {}
tasks = {}

ord_to_id = {}
id_to_ord = {}

for ido in range(len(data.get("orders"))):
    ord = data.get("orders")[ido]

    name = ord.get('name')
    o_tasks = ord.get('tasks')

    orders[name] = []

    ord_to_id[name] = "o" + str(ido)
    id_to_ord["o" + str(ido)] = name

    for idt in range(len(o_tasks)):
        tas = o_tasks[idt]
        task_name = "o" + str(ido) + "t" + str(idt)
        
        tasks[task_name] = tas.get("duration")
        machines[tas.get("machine")].append(task_name)
        orders[name].append(task_name)


In [76]:
print(orders)

{'charmander_figure::client_4710': ['o0t0', 'o0t1'], 'marketing_pin::client_0123': ['o1t0'], 'finish_order_412::client_3122': ['o2t0', 'o2t1', 'o2t2'], 'model_suite_prototype::client_8692': ['o3t0', 'o3t1']}


In [77]:
print(tasks)

{'o0t0': 5, 'o0t1': 2, 'o1t0': 4, 'o2t0': 6, 'o2t1': 2, 'o2t2': 1, 'o3t0': 4, 'o3t1': 3}


In [78]:
print("ID -> MAC", id_to_mac)
print("MAC -> ID", mac_to_id)
print("ID -> ORD", id_to_ord)
print("ORD -> ID", ord_to_id)

ID -> MAC {'m0': 'paint', 'm1': 'print_3d', 'm2': 'assembler'}
MAC -> ID {'paint': 'm0', 'print_3d': 'm1', 'assembler': 'm2'}
ID -> ORD {'o0': 'charmander_figure::client_4710', 'o1': 'marketing_pin::client_0123', 'o2': 'finish_order_412::client_3122', 'o3': 'model_suite_prototype::client_8692'}
ORD -> ID {'charmander_figure::client_4710': 'o0', 'marketing_pin::client_0123': 'o1', 'finish_order_412::client_3122': 'o2', 'model_suite_prototype::client_8692': 'o3'}


### Create literals for prolog

In [79]:
literals = []

for m in machines.items():
    l = "machineTasks(" + mac_to_id[m[0]] + ",["
    for t in m[1]:
        l += (t + ",")
    
    if len(m[1]) > 0: l = l[:-1]
    l += ("]).")

    literals.append(l)

In [80]:
for o in orders.items():
    l = "orders(" + ord_to_id[o[0]] + ",["
    for t in o[1]:
        l += (t + ",")
    
    if len(m[1]) > 0: l = l[:-1]
    l += ("]).")
    literals.append(l)

In [81]:
for t in tasks.items():
    l = "taskDuration(" + t[0] + "," + str(t[1]) + ")."
    literals.append(l)

In [82]:
sumHoresTotal = reduce(lambda x, acc: x + acc, tasks.values(), 0)

In [83]:
literals.append("maxHourInput(" + str(sumHoresTotal) + ").")

In [84]:
for l in literals: print(l)

machineTasks(m0,[o0t1,o2t1]).
machineTasks(m1,[o0t0,o1t0,o2t2,o3t0]).
machineTasks(m2,[o2t0,o3t1]).
orders(o0,[o0t0,o0t1]).
orders(o1,[o1t0]).
orders(o2,[o2t0,o2t1,o2t2]).
orders(o3,[o3t0,o3t1]).
taskDuration(o0t0,5).
taskDuration(o0t1,2).
taskDuration(o1t0,4).
taskDuration(o2t0,6).
taskDuration(o2t1,2).
taskDuration(o2t2,1).
taskDuration(o3t0,4).
taskDuration(o3t1,3).
maxHourInput(27).


In [85]:
for l in literals:
    plInput.write(l + "\n")

In [86]:
plInput.close()

### Read solution and get best

In [99]:
file_path = "./challenge1/"

raw = ""
with open(file_path + "prolog_output_challenge_1.txt", "r") as f:
    raw = reduce(lambda acc, x: acc + x + '\n', [line.strip() for line in f.readlines()], "")

#raw = open(file_path + "prolog_output_challenge_1.txt", "r")
raw_sol = raw.split("Unsatisfiable. So the optimal solution was this one with cost ")

### Prepare solution

In [100]:
if len(raw_sol) > 0: raw_sol = raw_sol[1]
else : 
    print("UNSAT!!")
    #return 0

raw_sol = raw_sol.split('\n\n%% END OF FOUND SOLUTION %%')[0]
cost, assig = raw_sol.split(':\n')

In [118]:
print(cost, assig)

14 Machine: m0
Task: o0t1 13-14
Task: o2t1 7-8
Machine: m1
Task: o0t0 5-9
Task: o1t0 10-13
Task: o2t2 14-14
Task: o3t0 1-4
Machine: m2
Task: o2t0 1-6
Task: o3t1 12-14



In [108]:
which_order = lambda tid: tid.split('t')[0]
which_task_number = lambda tid: int(tid.split('t')[1])

In [104]:
print(assig.split("Machine: "))

['', 'm0\nTask: o0t1 13-14\nTask: o2t1 7-8\n', 'm1\nTask: o0t0 5-9\nTask: o1t0 10-13\nTask: o2t2 14-14\nTask: o3t0 1-4\n', 'm2\nTask: o2t0 1-6\nTask: o3t1 12-14\n']


In [116]:
machines_res = []

for m in assig.split("Machine: ")[1:]:
    t = m.split('\n')
    dicMac = {"id" : t[0]}
    dicMac["tasks"] = []

    for tt in t[1:-1]:
        ttt = tt.split(' ')
        dicMac["tasks"].append(
            {
                "order": id_to_ord[which_order(ttt[1])],
                "task_number": which_task_number(ttt[1]),
                "start_at": int(ttt[2].split('-')[0]),
                "end_at": int(ttt[2].split('-')[1])
            }
        )
        #print(dicMac)
    
    machines_res.append(dicMac)


In [117]:
print(machines_res)

[{'id': 'm0', 'tasks': [{'order': 'charmander_figure::client_4710', 'task_number': 1, 'start_at': 13, 'end_at': 14}, {'order': 'finish_order_412::client_3122', 'task_number': 1, 'start_at': 7, 'end_at': 8}]}, {'id': 'm1', 'tasks': [{'order': 'charmander_figure::client_4710', 'task_number': 0, 'start_at': 5, 'end_at': 9}, {'order': 'marketing_pin::client_0123', 'task_number': 0, 'start_at': 10, 'end_at': 13}, {'order': 'finish_order_412::client_3122', 'task_number': 2, 'start_at': 14, 'end_at': 14}, {'order': 'model_suite_prototype::client_8692', 'task_number': 0, 'start_at': 1, 'end_at': 4}]}, {'id': 'm2', 'tasks': [{'order': 'finish_order_412::client_3122', 'task_number': 0, 'start_at': 1, 'end_at': 6}, {'order': 'model_suite_prototype::client_8692', 'task_number': 1, 'start_at': 12, 'end_at': 14}]}]


In [120]:
result_processed = {"cost" : cost, "assignation" : machines_res}
print(result_processed)

{'cost': '14', 'assignation': [{'id': 'm0', 'tasks': [{'order': 'charmander_figure::client_4710', 'task_number': 1, 'start_at': 13, 'end_at': 14}, {'order': 'finish_order_412::client_3122', 'task_number': 1, 'start_at': 7, 'end_at': 8}]}, {'id': 'm1', 'tasks': [{'order': 'charmander_figure::client_4710', 'task_number': 0, 'start_at': 5, 'end_at': 9}, {'order': 'marketing_pin::client_0123', 'task_number': 0, 'start_at': 10, 'end_at': 13}, {'order': 'finish_order_412::client_3122', 'task_number': 2, 'start_at': 14, 'end_at': 14}, {'order': 'model_suite_prototype::client_8692', 'task_number': 0, 'start_at': 1, 'end_at': 4}]}, {'id': 'm2', 'tasks': [{'order': 'finish_order_412::client_3122', 'task_number': 0, 'start_at': 1, 'end_at': 6}, {'order': 'model_suite_prototype::client_8692', 'task_number': 1, 'start_at': 12, 'end_at': 14}]}]}


### Write solution

## Challenge 2 parser

### Read and open files

In [132]:
file_path = './challenge2/'

In [133]:
# input reading
inJSON = open(file_path + "/" + "challenge_2_input.json")
data = json.load(inJSON)

# opening/creating output file
plInput = open("./challenge2/prolog_input_challenge_2.pl", "w")

In [134]:
print(data)

{'orders': [{'id': 'client::0000', 'quantity': 1, 'tasks': [{'task_number': 0, 'machine': 'painter'}, {'task_number': 1, 'machine': 'printer_fdm'}, {'task_number': 2, 'machine': 'hand_polish'}]}, {'id': 'client::0001', 'quantity': 2, 'tasks': [{'task_number': 0, 'machine': 'painter'}, {'task_number': 1, 'machine': 'printer_fdm'}, {'task_number': 2, 'machine': 'hand_polish'}]}, {'id': 'client::0002', 'quantity': 3, 'tasks': [{'task_number': 0, 'machine': 'painter'}, {'task_number': 1, 'machine': 'hand_polish'}]}, {'id': 'client::0003', 'quantity': 2, 'tasks': [{'task_number': 0, 'machine': 'painter'}, {'task_number': 1, 'machine': 'printer_fdm'}, {'task_number': 2, 'machine': 'hand_polish'}]}, {'id': 'client::0004', 'quantity': 1, 'tasks': [{'task_number': 0, 'machine': 'painter'}, {'task_number': 1, 'machine': 'printer_fdm'}, {'task_number': 2, 'machine': 'hand_polish'}]}, {'id': 'client::0005', 'quantity': 2, 'tasks': [{'task_number': 0, 'machine': 'printer_fdm'}, {'task_number': 1, '

### Machine data

In [135]:
machines = {}

mac_to_id = {} #Map a machine name to its id
id_to_mac = {} #Map a machine's id to its name

for idm in range(len(data.get("machines"))):
    m = data.get("machines")[idm]
    machines[m.get('id')] = []
    mac_to_id[m.get('id')] = "m" + str(idm)
    id_to_mac["m" + str(idm)] = m.get('id')

In [136]:
def linear_eq_params(eq):
    if eq == "":
        return 0, 0
    elif ' * x + ' in eq:
        return float(eq.split(' * x + ')[0]), float(eq.split(' * x + ')[1])
    elif ' * x' in eq:
        return float(eq.split(' * x')[0]), 0
    elif not ('x' in eq):
        return 0, float(eq)

In [137]:
machine_config = {}

for idm in range(len(data.get("machines"))):
    m = data.get("machines")[idm]

    MUL, ADD = linear_eq_params(m.get("execution_time"))
    machine_config[m.get('id')] = {
        "LT": float(m.get("load_time")),
        "UT": float(m.get("unload_time")),
        "MUL": MUL,
        "ADD": ADD,
        "MAXI": m.get("max_items")
    }


In [138]:
machine_config

{'printer_fdm': {'LT': 0.1, 'UT': 0.1, 'MUL': 1.2, 'ADD': 0, 'MAXI': 1},
 'printer_mjf': {'LT': 0.15, 'UT': 0.2, 'MUL': 0.3, 'ADD': 0.5, 'MAXI': 3},
 'hand_polish': {'LT': 0.0, 'UT': 0.0, 'MUL': 0.75, 'ADD': 0, 'MAXI': 1},
 'tumble_polish': {'LT': 0.15, 'UT': 0.3, 'MUL': 0, 'ADD': 1.0, 'MAXI': 4},
 'dyer': {'LT': 2.0, 'UT': 2.5, 'MUL': 0.8, 'ADD': 4.0, 'MAXI': 5},
 'painter': {'LT': 0.1, 'UT': 4.0, 'MUL': 1.2, 'ADD': 0, 'MAXI': 1}}

### Order data

In [142]:
orders = {}
tasks = {}

ord_to_id = {}
id_to_ord = {}

for ido in range(len(data.get("orders"))):
    ord = data.get("orders")[ido]

    name = ord.get('id')
    o_tasks = ord.get('tasks')
    qtt = ord.get('quantity')

    orders[name] = []

    ord_to_id[name] = "o" + str(ido)
    id_to_ord["o" + str(ido)] = name

    for tas in o_tasks:
        idt = int(tas.get("task_number"))
        task_name = "o" + str(ido) + "t" + str(idt)
        
        tasks[task_name] = qtt
        machines[tas.get("machine")].append(task_name)
        orders[name].append(task_name)

In [146]:
print(orders)

{'client::0000': ['o0t0', 'o0t1', 'o0t2'], 'client::0001': ['o1t0', 'o1t1', 'o1t2'], 'client::0002': ['o2t0', 'o2t1'], 'client::0003': ['o3t0', 'o3t1', 'o3t2'], 'client::0004': ['o4t0', 'o4t1', 'o4t2'], 'client::0005': ['o5t0', 'o5t1', 'o5t2'], 'client::0006': ['o6t0', 'o6t1'], 'client::0007': ['o7t0', 'o7t1'], 'client::0008': ['o8t0', 'o8t1', 'o8t2'], 'client::0009': ['o9t0', 'o9t1', 'o9t2'], 'client::0010': ['o10t0', 'o10t1', 'o10t2'], 'client::0011': ['o11t0', 'o11t1', 'o11t2', 'o11t3'], 'client::0012': ['o12t0', 'o12t1', 'o12t2'], 'client::0013': ['o13t0', 'o13t1'], 'client::0014': ['o14t0', 'o14t1', 'o14t2'], 'client::0015': ['o15t0'], 'client::0016': ['o16t0'], 'client::0017': ['o17t0', 'o17t1', 'o17t2'], 'client::0018': ['o18t0', 'o18t1'], 'client::0019': ['o19t0', 'o19t1']}


In [147]:
print(tasks)

{'o0t0': 1, 'o0t1': 1, 'o0t2': 1, 'o1t0': 2, 'o1t1': 2, 'o1t2': 2, 'o2t0': 3, 'o2t1': 3, 'o3t0': 2, 'o3t1': 2, 'o3t2': 2, 'o4t0': 1, 'o4t1': 1, 'o4t2': 1, 'o5t0': 2, 'o5t1': 2, 'o5t2': 2, 'o6t0': 1, 'o6t1': 1, 'o7t0': 2, 'o7t1': 2, 'o8t0': 2, 'o8t1': 2, 'o8t2': 2, 'o9t0': 3, 'o9t1': 3, 'o9t2': 3, 'o10t0': 1, 'o10t1': 1, 'o10t2': 1, 'o11t0': 3, 'o11t1': 3, 'o11t2': 3, 'o11t3': 3, 'o12t0': 1, 'o12t1': 1, 'o12t2': 1, 'o13t0': 1, 'o13t1': 1, 'o14t0': 3, 'o14t1': 3, 'o14t2': 3, 'o15t0': 2, 'o16t0': 1, 'o17t0': 1, 'o17t1': 1, 'o17t2': 1, 'o18t0': 1, 'o18t1': 1, 'o19t0': 2, 'o19t1': 2}


In [145]:
print("ID -> MAC", id_to_mac)
print("MAC -> ID", mac_to_id)
print("ID -> ORD", id_to_ord)
print("ORD -> ID", ord_to_id)

ID -> MAC {'m0': 'printer_fdm', 'm1': 'printer_mjf', 'm2': 'hand_polish', 'm3': 'tumble_polish', 'm4': 'dyer', 'm5': 'painter'}
MAC -> ID {'printer_fdm': 'm0', 'printer_mjf': 'm1', 'hand_polish': 'm2', 'tumble_polish': 'm3', 'dyer': 'm4', 'painter': 'm5'}
ID -> ORD {'o0': 'client::0000', 'o1': 'client::0001', 'o2': 'client::0002', 'o3': 'client::0003', 'o4': 'client::0004', 'o5': 'client::0005', 'o6': 'client::0006', 'o7': 'client::0007', 'o8': 'client::0008', 'o9': 'client::0009', 'o10': 'client::0010', 'o11': 'client::0011', 'o12': 'client::0012', 'o13': 'client::0013', 'o14': 'client::0014', 'o15': 'client::0015', 'o16': 'client::0016', 'o17': 'client::0017', 'o18': 'client::0018', 'o19': 'client::0019'}
ORD -> ID {'client::0000': 'o0', 'client::0001': 'o1', 'client::0002': 'o2', 'client::0003': 'o3', 'client::0004': 'o4', 'client::0005': 'o5', 'client::0006': 'o6', 'client::0007': 'o7', 'client::0008': 'o8', 'client::0009': 'o9', 'client::0010': 'o10', 'client::0011': 'o11', 'clien

### Create literals for prolog

In [158]:
literals = []

for m in machines.items():
    l = "machineTasks(" + mac_to_id[m[0]] + ",["
    for t in m[1]:
        l += (t + ",")
    
    if len(m[1]) > 0: l = l[:-1]
    l += ("]).")

    literals.append(l)

In [159]:
for o in orders.items():
    l = "orders(" + ord_to_id[o[0]] + ",["
    for t in o[1]:
        l += (t + ",")
    
    if len(m[1]) > 0: l = l[:-1]
    l += ("]).")
    literals.append(l)

In [160]:
for t in tasks.items():
    l = "taskDuration(" + t[0] + "," + str(t[1]) + ")."
    literals.append(l)

In [161]:
for mc in machine_config.items():
    l = "machineConfig(" + mc[0] + ","
    for param in mc[1].values():
        l += str(param) + ","
    l = l[:-1]
    l += ")."
    literals.append(l)

In [162]:
for l in literals: print(l)

machineTasks(m0,[o0t1,o1t1,o3t1,o4t1,o5t0,o6t1,o7t1,o8t0,o9t0,o10t2,o11t2,o12t2,o13t1,o14t2,o17t2,o18t1,o0t1,o1t1,o3t1,o4t1,o5t0,o6t1,o7t1,o8t0,o9t0,o10t2,o11t2,o12t2,o13t1,o14t2,o17t2,o18t1]).
machineTasks(m1,[o9t1,o11t0,o14t0,o9t1,o11t0,o14t0]).
machineTasks(m2,[o0t2,o1t2,o2t1,o3t2,o4t2,o5t1,o8t2,o10t0,o11t3,o12t0,o16t0,o17t1,o19t1,o0t2,o1t2,o2t1,o3t2,o4t2,o5t1,o8t2,o10t0,o11t3,o12t0,o16t0,o17t1,o19t1]).
machineTasks(m3,[]).
machineTasks(m4,[]).
machineTasks(m5,[o0t0,o1t0,o2t0,o3t0,o4t0,o5t2,o6t0,o7t0,o8t1,o9t2,o10t1,o11t1,o12t1,o13t0,o14t1,o15t0,o17t0,o18t0,o19t0,o0t0,o1t0,o2t0,o3t0,o4t0,o5t2,o6t0,o7t0,o8t1,o9t2,o10t1,o11t1,o12t1,o13t0,o14t1,o15t0,o17t0,o18t0,o19t0]).
orders(o0,[o0t0,o0t1,o0t2]).
orders(o1,[o1t0,o1t1,o1t2]).
orders(o2,[o2t0,o2t1]).
orders(o3,[o3t0,o3t1,o3t2]).
orders(o4,[o4t0,o4t1,o4t2]).
orders(o5,[o5t0,o5t1,o5t2]).
orders(o6,[o6t0,o6t1]).
orders(o7,[o7t0,o7t1]).
orders(o8,[o8t0,o8t1,o8t2]).
orders(o9,[o9t0,o9t1,o9t2]).
orders(o10,[o10t0,o10t1,o10t2]).
orders(o11,[

In [ ]:
for l in literals:
    plInput.write(l + "\n")

In [ ]:
plInput.close()